In [1]:
import dill
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
path_name = "../dataset/data/"

with open(path_name + 'comment-pos.data', 'rb') as file:
    datatofile = dill.load(file)

tagged_sents = []
for data in datatofile:
    text_inside = []
    for word, pos, label in data:
        text_inside.append((word, label))
    tagged_sents.append(text_inside)

train_sents, test_sents = train_test_split(tagged_sents, test_size=0.2, random_state=42)
print(len(train_sents))
print(len(test_sents))
print(train_sents[1])

552
138
[('ถ้า', 'B-c'), ('เดินทาง', 'I-c'), ('กลางคืน', 'I-c'), ('ก็', 'I-c'), ('รถทัวร์', 'I-c'), ('ครับ', 'I-c'), (' ', 'I-c'), (' ', 'O'), ('เพราะ', 'B-p'), ('รถ', 'I-p'), ('ไม่', 'I-p'), ('เยอะ', 'I-p'), (' ', 'I-p'), ('ความเสี่ยง', 'I-p'), ('การ', 'I-p'), ('เกิด', 'I-p'), ('อุบัติ', 'I-p'), ('ห', 'I-p'), ('ตุ', 'I-p'), ('ก็', 'I-p'), ('น้อย', 'I-p'), ('(', 'I-p'), ('มั้ง', 'I-p'), (')', 'I-p'), (' ', 'I-p'), (' ', 'O'), ('ถ้า', 'B-c'), ('กลางวัน', 'I-c'), ('ก็', 'I-c'), ('เครื่องบิน', 'I-c'), ('ครับ', 'I-c'), (' ', 'I-c'), (' ', 'O'), ('เพราะ', 'B-p'), (' ', 'I-p'), ('มัน', 'I-p'), ('ใช้เวลา', 'I-p'), ('น้อย', 'I-p'), ('จะ', 'I-p'), ('ได้', 'I-p'), ('มี', 'I-p'), ('เวลา', 'I-p'), ('ระหว่าง', 'I-p'), ('วัน', 'I-p'), ('เยอะ', 'I-p'), ('ๆ', 'I-p'), (' ', 'I-p')]


In [3]:
_NER_TAGS = [
        "O",
        "B_C",
        "B_P",
        "I_C",
        "I_P"
    ]

In [4]:
def convert_to_simple_transformer_format(sentences):
    sentence_id = []
    words = []
    labels = []

    for idx, sents in enumerate(sentences):
        for word, label in sents:
            label = label.upper().replace("-", "_")
            sentence_id.append(idx)
            words.append(word)
            labels.append(label)
    return pd.DataFrame(
        {"sentence_id": sentence_id, "words": words, "labels": labels}
    )    
            


In [5]:
train_ = convert_to_simple_transformer_format(train_sents)
train_

,sentence_id,words,labels
0,0,อะไหล่,B_C
1,0,เทอร์โบ,I_C
2,0,,I_C
3,0,อี,I_C
4,0,ซุ,I_C
...,...,...,...
37405,551,ทำ,I_P
37406,551,อะไร,I_P
37407,551,ได้,I_P
37408,551,หลายอย่าง,I_P


In [6]:
test_ = convert_to_simple_transformer_format(test_sents)

In [7]:
import torch
from simpletransformers.ner import NERModel, NERArgs

# Configure the model
ner_args = NERArgs()
ner_args.train_batch_size = 12
ner_args.evaluate_during_training = False
ner_args.overwrite_output_dir = True
ner_args.num_train_epochs = 100 #10


model = NERModel(
    "bert", "monsoon-nlp/bert-base-thai", args=ner_args, use_cuda=torch.cuda.is_available(), labels=_NER_TAGS
)

# Train the model
model.train_model(train_)

Some weights of the model checkpoint at monsoon-nlp/bert-base-thai were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
result, model_outputs, preds_list = model.eval_model(test_)
result

Running Evaluation: 100%|██████████| 18/18 [00:02<00:00,  7.04it/s]
C:\Users\pitiw\miniconda3\envs\argument\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_C seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\pitiw\miniconda3\envs\argument\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: I_C seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\pitiw\miniconda3\envs\argument\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_P seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\pitiw\miniconda3\envs\argument\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: I_P seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 2.5283145805199942,
 'precision': 0.069164265129683,
 'recall': 0.23529411764705882,
 'f1_score': 0.1069042316258352}

In [ ]:
idx = 60
test_pred = " ".join(list(map(lambda word: word[0], test_sents[idx])))
print(test_pred)
print(test_sents[idx])

ดู รายละเอียด งาน ดีกว่า ครับ   สำหรับ ผม   ผม ว่า   เซเว่น   หนัก เอาเรื่อง อยู่   ถ้า เป็น ผม เอง นะ   ผม เลือก ร้า นอ เม ซอน ครับ     เพราะ รายละเอียด งาน ไม่ เยอะ   ไม่ จุกจิก มาก   ถ้า เซเว่น   ปม ว่า น่าจะ ต้อง ทำงาน เป็น กะ   และ ไหนจะ เช็ค ของ   เช็ค สต๊อก   อื่นๆ อีก   เห็น พนักงาน มี บ่น เยอะ เลย ครับ   ต้อง รับผิดชอบ สินค้า อะไร แบบนี้ ด้วย   คือ ผม มองว่า ปัญหา จุกจิก มัน เยอะ กว่า ร้า นอ เม ซอน นะ   (คห สต. )  
[('ดู', 'O'), ('รายละเอียด', 'O'), ('งาน', 'O'), ('ดีกว่า', 'O'), ('ครับ', 'O'), (' ', 'O'), ('สำหรับ', 'O'), ('ผม', 'O'), (' ', 'O'), ('ผม', 'O'), ('ว่า', 'O'), (' ', 'O'), ('เซเว่น', 'O'), (' ', 'O'), ('หนัก', 'O'), ('เอาเรื่อง', 'O'), ('อยู่', 'O'), (' ', 'O'), ('ถ้า', 'O'), ('เป็น', 'O'), ('ผม', 'O'), ('เอง', 'O'), ('นะ', 'O'), (' ', 'O'), ('ผม', 'B-c'), ('เลือก', 'I-c'), ('ร้า', 'I-c'), ('นอ', 'I-c'), ('เม', 'I-c'), ('ซอน', 'I-c'), ('ครับ', 'I-c'), (' ', 'I-c'), (' ', 'O'), ('เพราะ', 'B-p'), ('รายละเอียด', 'I-p'), ('งาน', 'I-p'), ('ไม่', 'I-p'), ('เยอะ', 'I-p')

In [ ]:
# Make predictions with the model
predictions, raw_outputs = model.predict([test_pred])
print(predictions[0]) 

Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  4.81it/s]

[{'ดู': 'B_C'}, {'รายละเอียด': 'I_C'}, {'งาน': 'I_C'}, {'ดีกว่า': 'I_C'}, {'ครับ': 'I_C'}, {'สำหรับ': 'I_C'}, {'ผม': 'I_C'}, {'ผม': 'I_C'}, {'ว่า': 'I_C'}, {'เซเว่น': 'I_C'}, {'หนัก': 'I_C'}, {'เอาเรื่อง': 'I_C'}, {'อยู่': 'I_C'}, {'ถ้า': 'I_C'}, {'เป็น': 'I_C'}, {'ผม': 'I_C'}, {'เอง': 'I_C'}, {'นะ': 'I_C'}, {'ผม': 'I_C'}, {'เลือก': 'I_C'}, {'ร้า': 'I_C'}, {'นอ': 'I_P'}, {'เม': 'I_C'}, {'ซอน': 'I_C'}, {'ครับ': 'I_C'}, {'เพราะ': 'B_P'}, {'รายละเอียด': 'I_P'}, {'งาน': 'I_P'}, {'ไม่': 'I_P'}, {'เยอะ': 'I_P'}, {'ไม่': 'I_P'}, {'จุกจิก': 'I_P'}, {'มาก': 'I_P'}, {'ถ้า': 'I_P'}, {'เซเว่น': 'I_P'}, {'ปม': 'I_P'}, {'ว่า': 'I_P'}, {'น่าจะ': 'I_P'}, {'ต้อง': 'I_P'}, {'ทำงาน': 'I_P'}, {'เป็น': 'I_P'}, {'กะ': 'I_P'}, {'และ': 'I_P'}, {'ไหนจะ': 'I_P'}, {'เช็ค': 'I_P'}, {'ของ': 'I_P'}, {'เช็ค': 'I_P'}, {'สต๊อก': 'I_P'}, {'อื่นๆ': 'I_P'}, {'อีก': 'I_P'}, {'เห็น': 'I_P'}, {'พนักงาน': 'I_P'}, {'มี': 'I_P'}, {'บ่น': 'I_P'}, {'เยอะ': 'I_P'}, {'เลย': 'I_P'}, {'ครับ': 'O'}, {'ต้อง': 'O'}, {'รับผิดชอบ': 'I_P'